In [24]:
import numpy as np
from scipy.optimize import differential_evolution, NonlinearConstraint, root_scalar, minimize
from openpyxl import load_workbook
from consts_and_funcs import *
import matplotlib.pyplot as plt
# Global list to record optimization history
optimization_history = []
# Function to calculate shielding time (modified to record intervals for visualization)
def my_shield_time(theta, v, t_deploy1, delay1, t_deploy2, delay2, t_deploy3, delay3, record_intervals=False):
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit

    deploy_positions = []
    explode_positions = []
    explode_times = []
    for t_deploy, delay in [(t_deploy1, delay1), (t_deploy2, delay2), (t_deploy3, delay3)]:
        deploy_pos = FY1_INIT + drone_vel * t_deploy
        explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
        explode_t = t_deploy + delay
        deploy_positions.append(deploy_pos)
        explode_positions.append(explode_pos)
        explode_times.append(explode_t)

    # Cloud center function for each bomb
    def get_cloud_center(i):
        def cloud_center(t):
            tau = t - explode_times[i]
            if tau < 0:
                return explode_positions[i] + np.array([0, 0, -ESCAPE_VEL * tau])
            if tau > DURATION:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
            else:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * tau])
        return cloud_center

    # Distance difference function
    def min_distance_diff(t, cylinder_radius):
        min_distance = []
        for i in range(3):
            cloud_center = get_cloud_center(i)
            min_distance.append(get_distance_diff(cloud_center, cylinder_radius, 1)(t))
        return min(min_distance)

    # Calculate total shielding time for a given radius, and optionally record intervals
    def calculate_shield_time(cylinder_radius, record_intervals=False):
        def diff(t):
            return min_distance_diff(t, cylinder_radius)

        t_max = M1_TIME
        t = min(explode_times)
        total_time = 0.0
        intervals = [] # Record shielding intervals
        while t < t_max:
            while t < t_max and diff(t) > 0:
                t += 0.1
            if t >= t_max:
                break
            t_start = t
            while t < t_max and diff(t) <= 0:
                t += 0.1
            t_end = t
            if t_start != min(explode_times):
                root1 = root_scalar(diff, bracket=[t_start - 0.1, t_start], method='brentq')
                t1 = root1.root if root1.converged else t_start
            else:
                t1 = min(explode_times)
            if t_end < t_max:
                root2 = root_scalar(diff, bracket=[t_start, t_end], method='brentq')
                t2 = root2.root if root2.converged else t_end - 0.1
            else:
                t2 = t_max
            total_time += max(0, t2 - t1)
            if record_intervals:
                intervals.append((t1, t2))
        if record_intervals:
            return total_time, intervals
        return total_time

    inner = calculate_shield_time(INNER_RADIUS)
    outer = calculate_shield_time(OUTER_RADIUS)
    avg_time = (inner + outer) / 2

    if record_intervals:
        _, inner_intervals = calculate_shield_time(INNER_RADIUS, True)
        _, outer_intervals = calculate_shield_time(OUTER_RADIUS, True)
        # For simplicity, use average intervals or union; here we approximate with inner for viz
        return avg_time, inner_intervals

    return avg_time

# 计算偏心的屏蔽时间函数 (unchanged, but can add record if needed)
def par_shield_time(theta, v, t_deploy, delay):
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    # 部署位置
deploy_pos = FY1_INIT + drone_vel * t_deploy

# 爆炸位置（烟雾弹具有无人机的水平速度，加上重力）
explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
explode_t = t_deploy + delay

# 云中心函数
def cloud_center(t):
    tau = t - explode_t
    if tau < 0:
        return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
    if tau > DURATION:
        return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
    else:
        return explode_pos + np.array([0, 0, -SINK_SPEED * tau])

# 内半径和外半径的平均屏蔽时间
calculate_shield_time = get_calculate_shield_time(cloud_center, explode_t, 1)
inner = calculate_shield_time(INNER_RADIUS)
outer = calculate_shield_time(OUTER_RADIUS)
return (inner + outer) / 2

def my_min_distance(theta, v, t_deploy1, delay1, t_deploy2, delay2, t_deploy3, delay3):
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit

    deploy_positions = []
    explode_positions = []
    explode_times = []
    for t_deploy, delay in [(t_deploy1, delay1), (t_deploy2, delay2), (t_deploy3, delay3)]:
        deploy_pos = FY1_INIT + drone_vel * t_deploy
        explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
        explode_t = t_deploy + delay
        deploy_positions.append(deploy_pos)
        explode_positions.append(explode_pos)
        explode_times.append(explode_t)

    # Cloud center function for each bomb
    def get_cloud_center(i):
        def cloud_center(t):
            tau = t - explode_times[i]
            if tau < 0:
                return explode_positions[i] + np.array([0, 0, -ESCAPE_VEL * tau])
            if tau > DURATION:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
            else:
                return explode_positions[i] + np.array([0, 0, -SINK_SPEED * tau])
        return cloud_center

    # 粗略计算三个云团最小距离
    def min_distance_diff_rough(t):
        min_distance = []
        for i in range(3):
            cloud_center = get_cloud_center(i)
            min_distance.append(get_distance_diff_rough(cloud_center, 1)(t))
        return min(min_distance)

    # 计算某个时间段里的三个云团最小距离
    def min_distance(explode_t):
        t_max = M1_TIME
        # 如果爆炸太晚，直接返回
        if t_max - explode_t <= 0.1:
            return min_distance_diff_rough(min(t_max, explode_t))
        t_arange = np.arange(explode_t, t_max, 0.1)
        distance_arange = np.array([min_distance_diff_rough(t) for t in t_arange])
        return distance_arange.min()
    return min_distance(min(explode_times))

def par_min_distance(theta, v, t_deploy, delay):
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit

    # 部署位置
    deploy_pos = FY1_INIT + drone_vel * t_deploy

    # 爆炸位置（烟雾弹具有无人机的水平速度，加上重力）
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay

    # 云中心函数
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])

    return get_min_distance(cloud_center, explode_t, 1)

def single_bomb_shield_time(theta, v, t_deploy, delay):
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    # 第二步：使用第一步的结果作为初始种群再次运行differential_evolution
result = differential_evolution(
    objective,
    bounds,
    strategy='best1bin', # 可根据需要调整策略
    constraints=constraints,
    maxiter=300, # 可增加迭代次数以进一步优化
    popsize=20, # 设置为20，与初始种群大小匹配
    tol=1e-3,
    callback=callback,
    init=initial_population, # 使用第一步的结果作为初始种群
    disp=False,
    polish=True
)

# Callback for minimize to continue recording history
def minimize_callback(xk):
    current_time = -objective(xk)
    optimization_history.append(current_time)

opt_theta, opt_v, opt_t_deploy1, opt_delay1, opt_t_deploy2, opt_delay2, opt_t_deploy3, opt_delay3 = result.x
opt_time, shielding_intervals = my_shield_time(opt_theta, opt_v, opt_t_deploy1, opt_delay1, opt_t_deploy2, opt_delay2, opt_t_deploy3, opt_delay3, record_intervals=True)

# Calculate positions
dir_unit = np.array([-np.cos(opt_theta), np.sin(opt_theta), 0])
drone_vel = opt_v * dir_unit
deploy_pos1 = FY1_INIT + drone_vel * opt_t_deploy1
explode_pos1 = deploy_pos1 + drone_vel * opt_delay1 + np.array([0, 0, -0.5 * G * opt_delay1**2])
deploy_pos2 = FY1_INIT + drone_vel * opt_t_deploy2
explode_pos2 = deploy_pos2 + drone_vel * opt_delay2 + np.array([0, 0, -0.5 * G * opt_delay2**2])
deploy_pos3 = FY1_INIT + drone_vel * opt_t_deploy3
explode_pos3 = deploy_pos3 + drone_vel * opt_delay3 + np.array([0, 0, -0.5 * G * opt_delay3**2])

# Convert theta to degrees (0~360°, counterclockwise from x-axis)
theta_deg = np.degrees(opt_theta) % 360

file_path = 'A题/附件/result1.xlsx'
workbook = load_workbook(file_path)

# 选择工作表（默认第一个sheet，也可以用sheet名称）
sheet = workbook.active # 或者 workbook["Sheet1"]

for row in range(2, 5):
    sheet.cell(row=row, column=1).value = round(theta_deg, 3)
    sheet.cell(row=row, column=2).value = round(opt_v, 3)
    deploy_pos = [deploy_pos1, deploy_pos2, deploy_pos3][row - 2]
    explode_pos = [explode_pos1, explode_pos2, explode_pos3][row - 2]
    opt_t_deploy = [opt_t_deploy1, opt_t_deploy2, opt_t_deploy3][row - 2]
    opt_delay = [opt_delay1, opt_delay2, opt_delay3][row - 2]
    sheet.cell(row=row, column=4).value = round(deploy_pos[0], 3)
    sheet.cell(row=row, column=5).value = round(deploy_pos[1], 3)
    sheet.cell(row=row, column=6).value = round(deploy_pos[2], 3)
    sheet.cell(row=row, column=7).value = round(explode_pos[0], 3)
    sheet.cell(row=row, column=8).value = round(explode_pos[1], 3)
    sheet.cell(row=row, column=9).value = round(explode_pos[2], 3)
    sheet.cell(row=row, column=10).value = round(single_bomb_shield_time(opt_theta, opt_v, opt_t_deploy, opt_delay), 3)

# 保存文件
workbook.save(file_path)
print(f"已将内容写入 {file_path}.")

# Output results to console and txt file
final_output = (
    f"优化结果：\n"
    f"无人机运动方向: {theta_deg:.3f}°\n"
    f"无人机运动速度: {opt_v:.3f} m/s\n"
    f"无人机速度向量: {drone_vel}\n"
    f"干扰弹1: 投放时间: {opt_t_deploy1:.3f} s, 起爆延迟: {opt_delay1:.3f} s\n"
    f" 投放位置: {deploy_pos1}\n"
    f" 起爆位置: {explode_pos1}\n"
    f"干扰弹2: 投放时间: {opt_t_deploy2:.3f} s, 起爆延迟: {opt_delay2:.3f} s\n"
    f" 投放位置: {deploy_pos2}\n"
    f" 起爆位置: {explode_pos2}\n"
    f"干扰弹3: 投放时间: {opt_t_deploy3:.3f} s, 起爆延迟: {opt_delay3:.3f} s\n"
    f" 投放位置: {deploy_pos3}\n"
    f" 起爆位置: {explode_pos3}\n"
    f"总有效干扰时长(平均): {opt_time:.3f} s\n"
)
print(final_output.strip())
with open(log_file, 'a', encoding='utf-8') as f:
    f.write(final_output + "\n")

# Visualization 1: Motion trajectories schematic (3D plot for trajectories)
fig1 = plt.figure(figsize=(12, 9))
ax1 = fig1.add_subplot(111, projection='3d')

# Missile M1 trajectory: from initial pos to fake target (0,0,0)
m1_init = np.array([20000, 0, 2000], dtype=np.float64)
direction_to_fake = np.array([0, 0, 0], dtype=np.float64) - m1_init
direction_to_fake /= np.linalg.norm(direction_to_fake)
missile_speed = 300
t_missile = np.linspace(0, M1_TIME, 100)
m1_traj = m1_init + t_missile[:, np.newaxis] * direction_to_fake * missile_speed
ax1.plot(m1_traj[:, 0], m1_traj[:, 1], m1_traj[:, 2], 'r-', label='M1 Trajectory', linewidth=2)
# Drone FY1 trajectory: straight line from init, up to max deploy time
t_max_drone = max(opt_t_deploy1, opt_t_deploy2, opt_t_deploy3) + 5
t_drone = np.linspace(0, t_max_drone, 100)
fy1_traj = FY1_INIT + t_drone[:, np.newaxis] * drone_vel
ax1.plot(fy1_traj[:, 0], fy1_traj[:, 1], fy1_traj[:, 2], 'b-', label='FY1 Trajectory', linewidth=2)

# Bomb trajectories: parabolic from deploy to explode
times1 = np.linspace(opt_t_deploy1, opt_t_deploy1 + opt_delay1, 20)
gravity_term1 = np.zeros((20, 3), dtype=np.float64)
gravity_term1[:, 2] = -0.5 * G * (times1 - opt_t_deploy1)**2
bomb_traj1 = FY1_INIT + times1[:, np.newaxis] * drone_vel + gravity_term1
ax1.plot(bomb_traj1[:, 0], bomb_traj1[:, 1], bomb_traj1[:, 2], 'g--', label='Bomb 1 Trajectory', alpha=0.7)

times2 = np.linspace(opt_t_deploy2, opt_t_deploy2 + opt_delay2, 20)
gravity_term2 = np.zeros((20, 3), dtype=np.float64)
gravity_term2[:, 2] = -0.5 * G * (times2 - opt_t_deploy2)**2
bomb_traj2 = FY1_INIT + times2[:, np.newaxis] * drone_vel + gravity_term2
ax1.plot(bomb_traj2[:, 0], bomb_traj2[:, 1], bomb_traj2[:, 2], 'g--', label='Bomb 2 Trajectory', alpha=0.7)

times3 = np.linspace(opt_t_deploy3, opt_t_deploy3 + opt_delay3, 20)
gravity_term3 = np.zeros((20, 3), dtype=np.float64)
gravity_term3[:, 2] = -0.5 * G * (times3 - opt_t_deploy3)**2
bomb_traj3 = FY1_INIT + times3[:, np.newaxis] * drone_vel + gravity_term3
ax1.plot(bomb_traj3[:, 0], bomb_traj3[:, 1], bomb_traj3[:, 2], 'g--', label='Bomb 3 Trajectory', alpha=0.7)

# Explode points
ax1.scatter([explode_pos1[0], explode_pos2[0], explode_pos3[0]],
            [explode_pos1[1], explode_pos2[1], explode_pos3[1]],
            [explode_pos1[2], explode_pos2[2], explode_pos3[2]],
            c='orange', s=50, label='Explode Points')

# Fake target and true target cylinder
ax1.scatter(0, 0, 0, c='black', s=100, label='Fake Target')
theta_cyl = np.linspace(0, 2 * np.pi, 50)
cyl_x = 7 * np.cos(theta_cyl)
cyl_y = 200 + 7 * np.sin(theta_cyl)
ax1.plot(cyl_x, cyl_y, zs=0, zdir='z', color='purple', label='True Target Base')
ax1.plot(cyl_x, cyl_y, zs=10, zdir='z', color='purple')
for i in range(0, len(theta_cyl), 10):
    ax1.plot([cyl_x[i], cyl_x[i]], [cyl_y[i], cyl_y[i]], [0, 10], color='purple', alpha=0.5)

ax1.scatter(FY1_INIT[0], FY1_INIT[1], FY1_INIT[2], c='blue', s=100, label='FY1 Init')
ax1.scatter(m1_init[0], m1_init[1], m1_init[2], c='red', s=100, label='M1 Init')

ax1.set_xlabel('X (m)')
ax1.set_ylabel('Y (m)')
ax1.set_zlabel('Z (m)')
ax1.set_title('Motion Trajectories Schematic')
ax1.legend()
plt.savefig('trajectories.png')
plt.show()

# Visualization 2: Shielding time distribution schematic (time intervals)
fig2, ax2 = plt.subplots(figsize=(10, 6))
t_axis = np.linspace(0, M1_TIME, 1000)
shielding_status = np.zeros_like(t_axis)
for start, end in shielding_intervals:
    mask = (t_axis >= start) & (t_axis <= end)
    shielding_status[mask] = 1

ax2.fill_between(t_axis, 0, shielding_status, alpha=0.7, color='green', label='Shielding Periods')
ax2.plot(t_axis, shielding_status, 'g-', linewidth=2)
ax2.set_ylim(-0.1, 1.1)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Shielding Status')
ax2.set_title('Shielding Time Distribution')
ax2.legend()
explode_times = [opt_t_deploy1 + opt_delay1, opt_t_deploy2 + opt_delay2, opt_t_deploy3 + opt_delay3]
for et in explode_times:
    ax2.axvline(x=et, color='orange', linestyle='--', alpha=0.5, label='Explode Time' if et == explode_times[0] else "")
plt.savefig('shielding_distribution.png')
plt.show()

# Visualization 2: Three-view diagram (Front, Side, Top views)
fig2, (ax_front, ax_side, ax_top) = plt.subplots(1, 3, figsize=(18, 6))
# Front view (X-Z plane)
ax_front.plot(m1_traj[:, 0], m1_traj[:, 2], 'r-', label='M1 Trajectory', linewidth=2)
ax_front.plot(fy1_traj[:, 0], fy1_traj[:, 2], 'b-', label='FY1 Trajectory', linewidth=2)
ax_front.plot(bomb_traj1[:, 0], bomb_traj1[:, 2], 'g--', label='Bomb 1 Trajectory', alpha=0.7)
ax_front.plot(bomb_traj2[:, 0], bomb_traj2[:, 2], 'g--', label='Bomb 2 Trajectory', alpha=0.7)
ax_front.plot(bomb_traj3[:, 0], bomb_traj3[:, 2], 'g--', label='Bomb 3 Trajectory', alpha=0.7)
ax_front.scatter([explode_pos1[0], explode_pos2[0], explode_pos3[0]],
                [explode_pos1[2], explode_pos2[2], explode_pos3[2]],
                c='orange', s=50, label='Explode Points')
ax_front.scatter(FY1_INIT[0], FY1_INIT[2], c='blue', s=100, label='FY1 Init')
ax_front.scatter(m1_init[0], m1_init[2], c='red', s=100, label='M1 Init')
ax_front.scatter(0, 0, c='black', s=100, label='Fake Target')
ax_front.plot([7, 7], [0, 10], 'purple', label='True Target', alpha=0.5)
ax_front.plot([-7, -7], [0, 10], 'purple', alpha=0.5)
ax_front.set_xlabel('X (m)')
ax_front.set_ylabel('Z (m)')
ax_front.set_title('Front View (X-Z)')
ax_front.grid(True)
ax_front.legend()

# Side view (Y-Z plane)
ax_side.plot(fy1_traj[:, 1], fy1_traj[:, 2], 'b-', label='FY1 Trajectory', linewidth=2)
ax_side.plot(bomb_traj1[:, 1], bomb_traj1[:, 2], 'g--', label='Bomb 1 Trajectory', alpha=0.7)
ax_side.plot(bomb_traj2[:, 1], bomb_traj2[:, 2], 'g--', label='Bomb 2 Trajectory', alpha=0.7)
ax_side.plot(bomb_traj3[:, 1], bomb_traj3[:, 2], 'g--', label='Bomb 3 Trajectory', alpha=0.7)
ax_side.scatter([explode_pos1[1], explode_pos2[1], explode_pos3[1]],
                [explode_pos1[2], explode_pos2[2], explode_pos3[2]],
                c='orange', s=50, label='Explode Points')
ax_side.scatter(FY1_INIT[1], FY1_INIT[2], c='blue', s=100, label='FY1 Init')
ax_side.scatter(m1_init[1], m1_init[2], c='red', s=100, label='M1 Init')
ax_side.scatter(0, 0, c='black', s=100, label='Fake Target')
ax_side.plot([200+7, 200+7], [0, 10], 'purple', label='True Target', alpha=0.5)
ax_side.plot([200-7, 200-7], [0, 10], 'purple', alpha=0.5)
ax_side.set_xlabel('Y (m)')
ax_side.set_ylabel('Z (m)')
ax_side.set_title('Side View (Y-Z)')
ax_side.grid(True)
ax_side.legend()
# Top view (X-Y plane)
ax_top.plot(m1_traj[:, 0], m1_traj[:, 1], 'r-', label='M1 Trajectory', linewidth=2)
ax_top.plot(fy1_traj[:, 0], fy1_traj[:, 1], 'b-', label='FY1 Trajectory', linewidth=2)
ax_top.plot(bomb_traj1[:, 0], bomb_traj1[:, 1], 'g--', label='Bomb 1 Trajectory', alpha=0.7)
ax_top.plot(bomb_traj2[:, 0], bomb_traj2[:, 1], 'g--', label='Bomb 2 Trajectory', alpha=0.7)
ax_top.plot(bomb_traj3[:, 0], bomb_traj3[:, 1], 'g--', label='Bomb 3 Trajectory', alpha=0.7)
ax_top.scatter([explode_pos1[0], explode_pos2[0], explode_pos3[0]],
               [explode_pos1[1], explode_pos2[1], explode_pos3[1]],
               c='orange', s=50, label='Explode Points')
ax_top.scatter(FY1_INIT[0], FY1_INIT[1], c='blue', s=100, label='FY1 Init')
ax_top.scatter(m1_init[0], m1_init[1], c='red', s=100, label='M1 Init')
ax_top.scatter(0, 0, c='black', s=100, label='Fake Target')
theta_cyl = np.linspace(0, 2 * np.pi, 50)
cyl_x = 7 * np.cos(theta_cyl)
cyl_y = 200 + 7 * np.sin(theta_cyl)
ax_top.plot(cyl_x, cyl_y, 'purple', label='True Target', alpha=0.5)
ax_top.set_xlabel('X (m)')
ax_top.set_ylabel('Y (m)')
ax_top.set_title('Top View (X-Y)')
ax_top.grid(True)
ax_top.legend()

plt.tight_layout()
plt.savefig('three_view_diagram.png')
plt.show()

# Visualization 3: Iteration algorithm optimization results change (record every iteration)
if len(optimization_history) > 0:
    iterations = np.arange(1, len(optimization_history) + 1)
    fig3, ax3 = plt.subplots(figsize=(10, 6))
    ax3.plot(iterations, optimization_history, 'b-', label='Optimization Progress', linewidth=2)
    ax3.axhline(y=opt_time, color='red', linestyle='--', label=f'Final Shielding Time: {opt_time:.3f} s')
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Shielding Time (s)')
    ax3.set_title('Optimization Progress: Shielding Time vs Iteration')
    ax3.grid(True)
    ax3.legend()
    plt.savefig('optimization_progress.png')
    plt.show()
else:
    print("No optimization history recorded.")

print("Visualizations saved as PNG files: trajectories.png, three_view_diagram.png, optimization_progress.png")

ModuleNotFoundError: No module named 'consts_and_funcs'